# Explicación del Archivo `estructuras_juego.py`

Este archivo establece las estructuras fundamentales para representar y analizar juegos estratégicos usando teoría de juegos. Vamos a examinar cada componente del código detalladamente.

## Imports Iniciales

```python
from typing import Dict, List, Tuple, Set, Optional, Union
from pydantic import BaseModel, validator, Field
import itertools
import numpy as np
```

- `typing`: Proporciona anotaciones de tipo para mayor claridad y verificación estática
- `pydantic`: Marco para validación de datos y definición de modelos
- `itertools`: Utilizado para generar combinaciones de estrategias
- `numpy`: Proporciona funciones matemáticas y de álgebra lineal (aunque su uso es mínimo aquí)

## Clase `Estrategia`

```python
class Estrategia(BaseModel):
    nombre: str
    descripcion: Optional[str] = None
    
    def __hash__(self):
        return hash(self.nombre)
    
    def __eq__(self, other):
        if not isinstance(other, Estrategia):
            return False
        return self.nombre == other.nombre
```

Esta clase representa una posible acción o decisión que un jugador puede tomar.

- **Atributos**:
  - `nombre`: Identificador único de la estrategia (obligatorio)
  - `descripcion`: Explicación opcional de la estrategia
  
- **Métodos especiales**:
  - `__hash__`: Define cómo se calcula el hash del objeto para permitir su uso como clave en diccionarios
  - `__eq__`: Define la igualdad basada en el nombre de la estrategia

- **¿Por qué se implementó así?**
  - Al heredar de `BaseModel` obtenemos validación automática
  - Implementar `__hash__` y `__eq__` permite usar objetos `Estrategia` como claves en diccionarios y en operaciones de conjuntos
  - La igualdad basada en nombre simplifica comparaciones entre estrategias

## Clase `Pagos`

```python
class Pagos(BaseModel):
    valores: Dict[int, float]  # Mapeo de jugador a su pago
    
    def __getitem__(self, jugador: int) -> float:
        return self.valores[jugador]
```

Esta clase representa las recompensas que reciben los jugadores.

- **Atributos**:
  - `valores`: Diccionario que asigna a cada jugador (identificado por un entero) su pago (un valor float)
  
- **Métodos**:
  - `__getitem__`: Permite acceder a los pagos con la sintaxis `pagos[jugador]` en lugar de `pagos.valores[jugador]`

- **¿Por qué se implementó así?**
  - El diccionario proporciona acceso O(1) a los pagos por jugador
  - La implementación de `__getitem__` hace que el código sea más limpio y legible
  - Hereda de `BaseModel` para validación automática

## Clase `EstructuraDeJuego`

```python
class EstructuraDeJuego(BaseModel):
    nombre: str
    descripcion: Optional[str] = None
    jugadores: int = Field(gt=0)
    estrategias: Dict[int, List[Estrategia]]  # Mapeo de jugador a sus estrategias disponibles
    matriz_pagos: Dict[Tuple, Pagos]  # Mapeo de combinación de estrategias a pagos
```

Esta es la clase principal que define un juego completo. Encapsula todos los componentes necesarios para representar un juego en forma normal.

- **Atributos**:
  - `nombre`: Identificador del juego
  - `descripcion`: Texto explicativo opcional
  - `jugadores`: Número de participantes (debe ser positivo)
  - `estrategias`: Diccionario que mapea cada jugador a su lista de estrategias disponibles
  - `matriz_pagos`: Diccionario que asigna a cada combinación de estrategias los pagos resultantes

- **¿Por qué se implementó así?**
  - El uso de diccionarios permite representar juegos de cualquier número de jugadores y estrategias
  - La estructura de datos es intuitiva y refleja la representación matemática formal de juegos
  - `Field(gt=0)` garantiza que el número de jugadores sea positivo

### Validadores de `EstructuraDeJuego`

```python
@validator('estrategias')
def validar_jugadores_estrategias(cls, v, values):
    if 'jugadores' not in values:
        raise ValueError("Número de jugadores debe ser definido primero")
    
    # Verificar que hay estrategias para cada jugador
    if len(v) != values['jugadores']:
        raise ValueError(f"Debe definir estrategias para cada uno de los {values['jugadores']} jugadores")
    
    # Verificar que las claves son correctas
    if set(v.keys()) != set(range(values['jugadores'])):
        raise ValueError(f"Las claves de las estrategias deben ser de 0 a {values['jugadores']-1}")
    
    # Verificar que hay al menos una estrategia por jugador
    for jugador, estrategias in v.items():
        if len(estrategias) == 0:
            raise ValueError(f"El jugador {jugador} debe tener al menos una estrategia")
        
        # Verificar que no hay nombres duplicados en las estrategias de un jugador
        nombres = [e.nombre for e in estrategias]
        if len(nombres) != len(set(nombres)):
            raise ValueError(f"El jugador {jugador} tiene estrategias con nombres duplicados")
    
    return v

@validator('matriz_pagos')
def validar_matriz_pagos(cls, v, values):
    if 'jugadores' not in values or 'estrategias' not in values:
        raise ValueError("Jugadores y estrategias deben ser definidos primero")
    
    # Generar todas las combinaciones posibles de estrategias
    todas_combinaciones = set()
    for combo in itertools.product(*[
        [(jugador, i) for i, _ in enumerate(estrategias)]
        for jugador, estrategias in values['estrategias'].items()
    ]):
        # Convertir a estructura adecuada para llave del diccionario
        combo_key = tuple(sorted(combo))
        todas_combinaciones.add(combo_key)
    
    # Verificar que la matriz de pagos contiene todas las combinaciones
    if set(v.keys()) != todas_combinaciones:
        raise ValueError("La matriz de pagos debe contener todas las combinaciones posibles de estrategias")
    
    # Verificar que los pagos contienen a todos los jugadores
    for combo, pagos in v.items():
        if set(pagos.valores.keys()) != set(range(values['jugadores'])):
            raise ValueError(f"Los pagos para la combinación {combo} deben incluir a todos los jugadores")
    
    return v
```

Estos validadores garantizan que el juego esté bien definido matemáticamente.

- **Validador de estrategias**:
  - Verifica que hay estrategias definidas para cada jugador
  - Comprueba que los índices de jugadores son correctos (de 0 a n-1)
  - Asegura que cada jugador tenga al menos una estrategia
  - Verifica que no haya nombres duplicados de estrategias para un mismo jugador

- **Validador de matriz de pagos**:
  - Genera todas las combinaciones posibles de estrategias
  - Verifica que la matriz de pagos contenga exactamente estas combinaciones (ni más ni menos)
  - Comprueba que los pagos para cada combinación incluyan a todos los jugadores

- **¿Por qué estos validadores?**
  - Detectan errores comunes en la definición de juegos
  - Garantizan que la estructura cumpla con los requisitos matemáticos de teoría de juegos
  - Proporcionan mensajes de error informativos para depuración

### Métodos auxiliares de `EstructuraDeJuego`

```python
def obtener_estrategia(self, jugador: int, indice: int) -> Estrategia:
    """Obtiene una estrategia específica por jugador e índice"""
    return self.estrategias[jugador][indice]

def obtener_todas_combinaciones(self) -> List[Tuple[Tuple[int, int], ...]]:
    """Genera todas las combinaciones posibles de estrategias"""
    return list(itertools.product(*[
        [(jugador, i) for i in range(len(estrategias))]
        for jugador, estrategias in self.estrategias.items()
    ]))
```

Estos métodos facilitan el acceso a estrategias y generación de combinaciones.

- `obtener_estrategia`: Proporciona acceso directo a una estrategia específica
- `obtener_todas_combinaciones`: Genera todas las posibles combinaciones de estrategias en el juego

## Función `pagos_estrategia`

```python
def pagos_estrategia(juego: EstructuraDeJuego, estrategia_indices: List[int]) -> Pagos:
    """
    Calcula los pagos para una combinación específica de estrategias.
    
    Args:
        juego: La estructura del juego
        estrategia_indices: Lista de índices de estrategias, una por jugador
        
    Returns:
        Los pagos correspondientes a esa combinación de estrategias
    """
    if len(estrategia_indices) != juego.jugadores:
        raise ValueError(f"Debe especificar exactamente {juego.jugadores} estrategias")
    
    # Construir la clave para buscar en la matriz de pagos
    combo = tuple(sorted([(jugador, estrategia_indices[jugador]) for jugador in range(juego.jugadores)]))
    
    return juego.matriz_pagos[combo]
```

Esta función calcula los pagos resultantes de una combinación específica de estrategias.

- **Parámetros**:
  - `juego`: La estructura completa del juego
  - `estrategia_indices`: Lista de índices, donde cada índice representa la estrategia elegida por el jugador correspondiente

- **Funcionalidad**:
  - Verifica que se proporcionen estrategias para todos los jugadores
  - Construye la clave (tupla ordenada) para buscar en el diccionario de la matriz de pagos
  - Retorna el objeto `Pagos` correspondiente

- **¿Por qué se implementó así?**
  - Interfaz simple: solo se necesitan índices de estrategias en lugar de objetos completos
  - La conversión a tupla ordenada garantiza buscar correctamente en la matriz de pagos
  - Comprobaciones de validación para capturar errores temprano

## Función `encontrar_equilibrios_nash`

```python
def encontrar_equilibrios_nash(juego: EstructuraDeJuego) -> List[Tuple[List[Estrategia], Pagos]]:
    """
    Encuentra todos los equilibrios de Nash en el juego.
    
    Args:
        juego: La estructura del juego
        
    Returns:
        Lista de tuplas (estrategias, pagos) para cada equilibrio de Nash
    """
    equilibrios = []
    
    # Para cada combinación posible de estrategias
    for combo in juego.obtener_todas_combinaciones():
        es_equilibrio = True
        
        # Para cada jugador
        for jugador in range(juego.jugadores):
            estrategia_actual_idx = combo[jugador][1]
            pago_actual = pagos_estrategia(juego, [c[1] for c in sorted(combo, key=lambda x: x[0])]).valores[jugador]
            
            # Probar todas las estrategias alternativas para este jugador
            for alt_idx in range(len(juego.estrategias[jugador])):
                if alt_idx == estrategia_actual_idx:
                    continue
                
                # Crear una nueva combinación con la estrategia alternativa
                nueva_combo = list(combo)
                nueva_combo[jugador] = (jugador, alt_idx)
                nueva_combo = tuple(nueva_combo)
                
                # Calcular el pago con la estrategia alternativa
                nuevo_pago = pagos_estrategia(juego, [c[1] for c in sorted(nueva_combo, key=lambda x: x[0])]).valores[jugador]
                
                # Si hay un mejor pago con otra estrategia, no es equilibrio
                if nuevo_pago > pago_actual:
                    es_equilibrio = False
                    break
            
            if not es_equilibrio:
                break
        
        # Si es un equilibrio, agregarlo a la lista
        if es_equilibrio:
            estrategias_eq = [juego.obtener_estrategia(jugador, idx) for jugador, idx in sorted(combo, key=lambda x: x[0])]
            pagos_eq = pagos_estrategia(juego, [c[1] for c in sorted(combo, key=lambda x: x[0])])
            equilibrios.append((estrategias_eq, pagos_eq))
    
    return equilibrios
```

Esta función implementa el algoritmo para encontrar todos los equilibrios de Nash en un juego.

- **Parámetros**:
  - `juego`: La estructura completa del juego

- **Algoritmo**:
  1. Examina cada posible combinación de estrategias
  2. Para cada jugador, verifica si puede mejorar su pago cambiando unilateralmente su estrategia
  3. Si ningún jugador puede mejorar, la combinación es un equilibrio de Nash
  4. Agrega a la lista de equilibrios tanto las estrategias como los pagos

- **¿Por qué se implementó así?**
  - El enfoque de fuerza bruta es fácil de entender y garantiza encontrar todos los equilibrios
  - La estructura de resultados (estrategias y pagos) facilita el análisis posterior
  - El algoritmo refleja la definición formal de equilibrio de Nash: ningún jugador puede mejorar cambiando unilateralmente

## Resumen

Este archivo proporciona un marco flexible y bien definido para representar y analizar juegos estratégicos en teoría de juegos:

1. Define estructuras de datos claras para estrategias y pagos
2. Implementa validaciones para garantizar juegos matemáticamente correctos
3. Proporciona funciones para calcular pagos y encontrar equilibrios de Nash
4. Utiliza Pydantic para validación automática y tipos fuertes para mayor seguridad

La implementación es intencionalmente general para permitir representar diversos tipos de juegos y es extensible para análisis adicionales.

# Codigo de la clase base

In [1]:
from typing import Dict, List, Tuple, Set, Optional, Union
from pydantic import BaseModel, validator, Field
import itertools
import numpy as np

class Estrategia(BaseModel):
    nombre: str
    descripcion: Optional[str] = None
    
    def __hash__(self):
        return hash(self.nombre)
    
    def __eq__(self, other):
        if not isinstance(other, Estrategia):
            return False
        return self.nombre == other.nombre

class Pagos(BaseModel):
    valores: Dict[int, float]  # Mapeo de jugador a su pago
    
    def __getitem__(self, jugador: int) -> float:
        return self.valores[jugador]

class EstructuraDeJuego(BaseModel):
    nombre: str
    descripcion: Optional[str] = None
    jugadores: int = Field(gt=0)
    estrategias: Dict[int, List[Estrategia]]  # Mapeo de jugador a sus estrategias disponibles
    matriz_pagos: Dict[Tuple, Pagos]  # Mapeo de combinación de estrategias a pagos
    
    @validator('estrategias')
    def validar_jugadores_estrategias(cls, v, values):
        if 'jugadores' not in values:
            raise ValueError("Número de jugadores debe ser definido primero")
        
        # Verificar que hay estrategias para cada jugador
        if len(v) != values['jugadores']:
            raise ValueError(f"Debe definir estrategias para cada uno de los {values['jugadores']} jugadores")
        
        # Verificar que las claves son correctas
        if set(v.keys()) != set(range(values['jugadores'])):
            raise ValueError(f"Las claves de las estrategias deben ser de 0 a {values['jugadores']-1}")
        
        # Verificar que hay al menos una estrategia por jugador
        for jugador, estrategias in v.items():
            if len(estrategias) == 0:
                raise ValueError(f"El jugador {jugador} debe tener al menos una estrategia")
            
            # Verificar que no hay nombres duplicados en las estrategias de un jugador
            nombres = [e.nombre for e in estrategias]
            if len(nombres) != len(set(nombres)):
                raise ValueError(f"El jugador {jugador} tiene estrategias con nombres duplicados")
        
        return v
    
    @validator('matriz_pagos')
    def validar_matriz_pagos(cls, v, values):
        if 'jugadores' not in values or 'estrategias' not in values:
            raise ValueError("Jugadores y estrategias deben ser definidos primero")
        
        # Generar todas las combinaciones posibles de estrategias
        todas_combinaciones = set()
        for combo in itertools.product(*[
            [(jugador, i) for i, _ in enumerate(estrategias)]
            for jugador, estrategias in values['estrategias'].items()
        ]):
            # Convertir a estructura adecuada para llave del diccionario
            combo_key = tuple(sorted(combo))
            todas_combinaciones.add(combo_key)
        
        # Verificar que la matriz de pagos contiene todas las combinaciones
        if set(v.keys()) != todas_combinaciones:
            raise ValueError("La matriz de pagos debe contener todas las combinaciones posibles de estrategias")
        
        # Verificar que los pagos contienen a todos los jugadores
        for combo, pagos in v.items():
            if set(pagos.valores.keys()) != set(range(values['jugadores'])):
                raise ValueError(f"Los pagos para la combinación {combo} deben incluir a todos los jugadores")
        
        return v
    
    def obtener_estrategia(self, jugador: int, indice: int) -> Estrategia:
        """Obtiene una estrategia específica por jugador e índice"""
        return self.estrategias[jugador][indice]
    
    def obtener_todas_combinaciones(self) -> List[Tuple[Tuple[int, int], ...]]:
        """Genera todas las combinaciones posibles de estrategias"""
        return list(itertools.product(*[
            [(jugador, i) for i in range(len(estrategias))]
            for jugador, estrategias in self.estrategias.items()
        ]))

def pagos_estrategia(juego: EstructuraDeJuego, estrategia_indices: List[int]) -> Pagos:
    """
    Calcula los pagos para una combinación específica de estrategias.
    
    Args:
        juego: La estructura del juego
        estrategia_indices: Lista de índices de estrategias, una por jugador
        
    Returns:
        Los pagos correspondientes a esa combinación de estrategias
    """
    if len(estrategia_indices) != juego.jugadores:
        raise ValueError(f"Debe especificar exactamente {juego.jugadores} estrategias")
    
    # Construir la clave para buscar en la matriz de pagos
    combo = tuple(sorted([(jugador, estrategia_indices[jugador]) for jugador in range(juego.jugadores)]))
    
    return juego.matriz_pagos[combo]

def encontrar_equilibrios_nash(juego: EstructuraDeJuego) -> List[Tuple[List[Estrategia], Pagos]]:
    """
    Encuentra todos los equilibrios de Nash en el juego.
    
    Args:
        juego: La estructura del juego
        
    Returns:
        Lista de tuplas (estrategias, pagos) para cada equilibrio de Nash
    """
    equilibrios = []
    
    # Para cada combinación posible de estrategias
    for combo in juego.obtener_todas_combinaciones():
        es_equilibrio = True
        
        # Para cada jugador
        for jugador in range(juego.jugadores):
            estrategia_actual_idx = combo[jugador][1]
            pago_actual = pagos_estrategia(juego, [c[1] for c in sorted(combo, key=lambda x: x[0])]).valores[jugador]
            
            # Probar todas las estrategias alternativas para este jugador
            for alt_idx in range(len(juego.estrategias[jugador])):
                if alt_idx == estrategia_actual_idx:
                    continue
                
                # Crear una nueva combinación con la estrategia alternativa
                nueva_combo = list(combo)
                nueva_combo[jugador] = (jugador, alt_idx)
                nueva_combo = tuple(nueva_combo)
                
                # Calcular el pago con la estrategia alternativa
                nuevo_pago = pagos_estrategia(juego, [c[1] for c in sorted(nueva_combo, key=lambda x: x[0])]).valores[jugador]
                
                # Si hay un mejor pago con otra estrategia, no es equilibrio
                if nuevo_pago > pago_actual:
                    es_equilibrio = False
                    break
            
            if not es_equilibrio:
                break
        
        # Si es un equilibrio, agregarlo a la lista
        if es_equilibrio:
            estrategias_eq = [juego.obtener_estrategia(jugador, idx) for jugador, idx in sorted(combo, key=lambda x: x[0])]
            pagos_eq = pagos_estrategia(juego, [c[1] for c in sorted(combo, key=lambda x: x[0])])
            equilibrios.append((estrategias_eq, pagos_eq))
    
    return equilibrios

/tmp/ipykernel_5591/1181946185.py:31: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.7/migration/
  @validator('estrategias')
/tmp/ipykernel_5591/1181946185.py:56: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.7/migration/
  @validator('matriz_pagos')


# Explicación del Archivo `juego_dilema_prisionero.py`

Este archivo implementa una versión modificada del clásico "Dilema del Prisionero" utilizando nuestro framework de teoría de juegos. Vamos a analizar el código paso a paso para entender cómo se modeló este juego y qué significa cada componente.

## Imports Iniciales

```python
from estructuras_juego import EstructuraDeJuego, Estrategia, Pagos, encontrar_equilibrios_nash
```

- Importamos las clases y funciones básicas que definimos en nuestro marco de teoría de juegos
- Estas importaciones nos permiten definir las estrategias, estructura y analizar el juego

## Definición de Estrategias

```python
callar = Estrategia(
    nombre="Callar", 
    descripcion="Mantener silencio y no confesar el crimen"
)

confesar = Estrategia(
    nombre="Confesar", 
    descripcion="Confesar el crimen e implicar al otro sospechoso"
)

evidencia = Estrategia(
    nombre="Proporcionar Evidencia", 
    descripcion="Ofrecer evidencia que podría reducir la propia sentencia pero aumentar la del otro"
)
```

Aquí definimos las tres estrategias disponibles para los jugadores:

1. **Callar**: La estrategia cooperativa tradicional del dilema del prisionero
2. **Confesar**: La estrategia no cooperativa tradicional
3. **Proporcionar Evidencia**: Una nueva estrategia que añade complejidad al juego clásico

- **¿Por qué tres estrategias?** 
  - Esto extiende el juego clásico (que solo tiene dos estrategias) para mostrar cómo añadir opciones puede cambiar la dinámica del juego
  - La estrategia adicional representa un nivel intermedio de cooperación/no cooperación

## Creación de la Estructura del Juego

```python
dilema_prisionero = EstructuraDeJuego(
    nombre="Dilema del Prisionero Modificado",
    descripcion="""
    Dos sospechosos son interrogados por separado por un crimen. Cada uno tiene tres opciones:
    1. Callar: No confesar y mantener silencio
    2. Confesar: Confesar el crimen e implicar al otro sospechoso
    3. Proporcionar Evidencia: Ofrecer información que reduce la propia condena pero incrementa la del otro
    
    Los pagos representan años de libertad (valores más altos son mejores):
    - Si ambos callan, ambos reciben una sentencia menor (7 años de libertad cada uno)
    - Si uno confiesa y el otro calla, el que confiesa queda libre (10 años de libertad) 
      y el que calla recibe una sentencia severa (0 años de libertad)
    - Si ambos confiesan, ambos reciben una sentencia moderada (3 años de libertad cada uno)
    - Si uno proporciona evidencia y el otro calla, el que proporciona evidencia recibe una reducción 
      de sentencia (8 años de libertad) y el que calla recibe una sentencia más severa (-2 años de libertad, 
      representando más años en prisión por cargos adicionales)
    - Si uno proporciona evidencia y el otro confiesa, el que proporciona evidencia recibe 
      una sentencia menor (4 años de libertad) y el que confiesa una sentencia moderada (2 años de libertad)
    - Si ambos proporcionan evidencia, ambos reciben una sentencia grave (1 año de libertad cada uno)
    """,
    jugadores=2,
    estrategias={
        0: [callar, confesar, evidencia],
        1: [callar, confesar, evidencia]
    },
    matriz_pagos={
        # (jugador, índice_estrategia)
        tuple(sorted([(0, 0), (1, 0)])): Pagos(valores={0: 7, 1: 7}),  # Ambos callan
        tuple(sorted([(0, 0), (1, 1)])): Pagos(valores={0: 0, 1: 10}),  # J0 calla, J1 confiesa
        tuple(sorted([(0, 1), (1, 0)])): Pagos(valores={0: 10, 1: 0}),  # J0 confiesa, J1 calla
        tuple(sorted([(0, 1), (1, 1)])): Pagos(valores={0: 3, 1: 3}),  # Ambos confiesan
        
        tuple(sorted([(0, 0), (1, 2)])): Pagos(valores={0: -2, 1: 8}),  # J0 calla, J1 proporciona evidencia
        tuple(sorted([(0, 2), (1, 0)])): Pagos(valores={0: 8, 1: -2}),  # J0 proporciona evidencia, J1 calla
        
        tuple(sorted([(0, 1), (1, 2)])): Pagos(valores={0: 2, 1: 4}),  # J0 confiesa, J1 proporciona evidencia
        tuple(sorted([(0, 2), (1, 1)])): Pagos(valores={0: 4, 1: 2}),  # J0 proporciona evidencia, J1 confiesa
        
        tuple(sorted([(0, 2), (1, 2)])): Pagos(valores={0: 1, 1: 1}),  # Ambos proporcionan evidencia
    }
)
```

Esta es la definición completa del juego:

- **Metadatos**:
  - `nombre` y `descripcion`: Proporcionan contexto e historia para el juego
  - `jugadores`: Especifica que hay 2 jugadores (como en el dilema clásico)

- **Estrategias**:
  - Cada jugador tiene las mismas tres estrategias disponibles
  - El diccionario mapea cada jugador (0 y 1) a su lista de estrategias

- **Matriz de Pagos**:
  - Define los pagos para cada posible combinación de estrategias
  - Los pagos representan "años de libertad" (números más altos son mejores)
  - Clave: tuple(sorted([(jugador1, estrategia1), (jugador2, estrategia2)]))
  - Valor: Pagos(valores={jugador1: pago1, jugador2: pago2})

- **Estructura de la matriz de pagos**:
  1. **Combinaciones clásicas**:
     - Ambos callan: (7, 7) - Resultado cooperativo moderadamente bueno
     - Uno calla, otro confiesa: (0, 10) o (10, 0) - El que confiesa sale mejor
     - Ambos confiesan: (3, 3) - Peor que si ambos callaran, pero mejor que ser el único que calla
  
  2. **Nuevas combinaciones con "Proporcionar Evidencia"**:
     - Callar vs. Evidencia: (-2, 8) o (8, -2) - El que proporciona evidencia sale mejor, el otro peor que en cualquier otro caso
     - Confesar vs. Evidencia: (2, 4) o (4, 2) - Resultados intermedios
     - Ambos proporcionan evidencia: (1, 1) - El peor resultado colectivo

- **¿Por qué estos valores de pago?**
  - Preservan la estructura esencial del dilema del prisionero clásico
  - Añaden complejidad con la nueva estrategia "Proporcionar Evidencia"
  - Crean tensiones interesantes entre interés individual y colectivo

## Análisis de Equilibrios de Nash

```python
# Encontrar equilibrios de Nash
equilibrios = encontrar_equilibrios_nash(dilema_prisionero)

print(f"Juego: {dilema_prisionero.nombre}")
print("Equilibrios de Nash encontrados:")
for i, (estrategias, pagos) in enumerate(equilibrios, 1):
    print(f"\nEquilibrio {i}:")
    for j, estrategia in enumerate(estrategias):
        print(f"  Jugador {j}: {estrategia.nombre}")
    print("  Pagos:")
    for j in range(dilema_prisionero.jugadores):
        print(f"    Jugador {j}: {pagos.valores[j]}")
```

Esta parte del código:
1. Utiliza la función `encontrar_equilibrios_nash` para identificar todos los equilibrios
2. Imprime cada equilibrio encontrado mostrando:
   - Las estrategias de cada jugador
   - Los pagos resultantes

## Análisis e Interpretación de Resultados

```python
# Análisis de los equilibrios
print("\nAnálisis de los equilibrios:")

if not equilibrios:
    print("  No se encontraron equilibrios de Nash en este juego.")
else:
    for i, (estrategias, pagos) in enumerate(equilibrios, 1):
        print(f"\n  Equilibrio {i}:")
        estrategias_str = ", ".join([f"Jugador {j}: {e.nombre}" for j, e in enumerate(estrategias)])
        pagos_str = ", ".join([f"Jugador {j}: {p}" for j, p in pagos.valores.items()])
        print(f"  Estrategias: {estrategias_str}")
        print(f"  Pagos: {pagos_str}")
        
        # Análisis específico para este juego
        if all(e.nombre == "Confesar" for e in estrategias):
            print("  Este es el clásico equilibrio del dilema del prisionero: aunque ambos jugadores")
            print("  estarían mejor si cooperaran (callaran), la estrategia dominante es confesar.")
            print("  Este equilibrio es ineficiente desde el punto de vista de Pareto, ya que")
            print("  ambos jugadores podrían mejorar si cambiaran simultáneamente a 'Callar'.")
        elif all(e.nombre == "Proporcionar Evidencia" for e in estrategias):
            print("  En este equilibrio, ambos jugadores eligen proporcionar evidencia, lo que resulta")
            print("  en una situación aún peor que la confesión mutua. Esto ilustra cómo la introducción")
            print("  de opciones adicionales puede empeorar los resultados en juegos no cooperativos.")
```

Esta sección proporciona un análisis detallado de los equilibrios encontrados:

1. Verifica si se encontraron equilibrios
2. Para cada equilibrio:
   - Muestra un resumen de estrategias y pagos
   - Proporciona un análisis específico según los tipos de equilibrios

3. **Análisis del equilibrio clásico** (ambos confiesan):
   - Explica la paradoja central del dilema del prisionero
   - Señala la ineficiencia de Pareto (los jugadores podrían mejorar conjuntamente)

4. **Análisis del nuevo equilibrio** (ambos proporcionan evidencia):
   - Muestra cómo la nueva opción puede llevar a resultados aún peores
   - Ilustra cómo más opciones no siempre mejoran los resultados en juegos no cooperativos

## Resultados Esperados

Al ejecutar este código, deberíamos encontrar dos equilibrios de Nash:

1. **Ambos jugadores confiesan**: El equilibrio clásico del dilema del prisionero
   - Pagos: (3, 3)
   - Este es un equilibrio porque ningún jugador puede mejorar cambiando solo su estrategia
   - Es subóptimo porque ambos obtendrían mejores pagos (7, 7) si ambos callaran

2. **Ambos jugadores proporcionan evidencia**: Un nuevo equilibrio introducido por la estrategia adicional
   - Pagos: (1, 1)
   - Este es incluso peor que el equilibrio clásico
   - Ilustra cómo añadir opciones puede empeorar la situación en juegos no cooperativos

## Implicaciones y Aprendizajes

Este juego modificado extiende el dilema del prisionero clásico para mostrar:

1. **Dilemas sociales más complejos**: Con más opciones que el simple binario cooperar/no cooperar
2. **Equilibrios múltiples**: Más de un equilibrio de Nash, algunos peores que otros
3. **Escalada de no-cooperación**: Cómo la introducción de nuevas estrategias puede llevar a resultados aún más ineficientes

La estructura general refleja muchas situaciones reales donde:
- La cooperación beneficiaría a todos
- Los incentivos individuales llevan a resultados subóptimos
- La introducción de más opciones puede complicar aún más la situación

Este tipo de análisis es útil en economía, ciencias políticas, relaciones internacionales, y muchos otros campos donde se estudian los dilemas de acción colectiva.

# Codigo

In [2]:
# Definición del juego: Dilema del Prisionero Modificado
# En esta versión, dos sospechosos son interrogados por separado pero tienen la opción
# adicional de proporcionar evidencia que podría reducir su sentencia mientras incrementa
# la del otro. Esta modificación añade un nivel de complejidad al dilema clásico.

# Definimos las estrategias para ambos jugadores
callar = Estrategia(
    nombre="Callar", 
    descripcion="Mantener silencio y no confesar el crimen"
)

confesar = Estrategia(
    nombre="Confesar", 
    descripcion="Confesar el crimen e implicar al otro sospechoso"
)

evidencia = Estrategia(
    nombre="Proporcionar Evidencia", 
    descripcion="Ofrecer evidencia que podría reducir la propia sentencia pero aumentar la del otro"
)

# Creamos la estructura del juego
dilema_prisionero = EstructuraDeJuego(
    nombre="Dilema del Prisionero Modificado",
    descripcion="""
    Dos sospechosos son interrogados por separado por un crimen. Cada uno tiene tres opciones:
    1. Callar: No confesar y mantener silencio
    2. Confesar: Confesar el crimen e implicar al otro sospechoso
    3. Proporcionar Evidencia: Ofrecer información que reduce la propia condena pero incrementa la del otro
    
    Los pagos representan años de libertad (valores más altos son mejores):
    - Si ambos callan, ambos reciben una sentencia menor (7 años de libertad cada uno)
    - Si uno confiesa y el otro calla, el que confiesa queda libre (10 años de libertad) 
      y el que calla recibe una sentencia severa (0 años de libertad)
    - Si ambos confiesan, ambos reciben una sentencia moderada (3 años de libertad cada uno)
    - Si uno proporciona evidencia y el otro calla, el que proporciona evidencia recibe una reducción 
      de sentencia (8 años de libertad) y el que calla recibe una sentencia más severa (-2 años de libertad, 
      representando más años en prisión por cargos adicionales)
    - Si uno proporciona evidencia y el otro confiesa, el que proporciona evidencia recibe 
      una sentencia menor (4 años de libertad) y el que confiesa una sentencia moderada (2 años de libertad)
    - Si ambos proporcionan evidencia, ambos reciben una sentencia grave (1 año de libertad cada uno)
    """,
    jugadores=2,
    estrategias={
        0: [callar, confesar, evidencia],
        1: [callar, confesar, evidencia]
    },
    matriz_pagos={
        # (jugador, índice_estrategia)
        tuple(sorted([(0, 0), (1, 0)])): Pagos(valores={0: 7, 1: 7}),  # Ambos callan
        tuple(sorted([(0, 0), (1, 1)])): Pagos(valores={0: 0, 1: 10}),  # J0 calla, J1 confiesa
        tuple(sorted([(0, 1), (1, 0)])): Pagos(valores={0: 10, 1: 0}),  # J0 confiesa, J1 calla
        tuple(sorted([(0, 1), (1, 1)])): Pagos(valores={0: 3, 1: 3}),  # Ambos confiesan
        
        tuple(sorted([(0, 0), (1, 2)])): Pagos(valores={0: -2, 1: 8}),  # J0 calla, J1 proporciona evidencia
        tuple(sorted([(0, 2), (1, 0)])): Pagos(valores={0: 8, 1: -2}),  # J0 proporciona evidencia, J1 calla
        
        tuple(sorted([(0, 1), (1, 2)])): Pagos(valores={0: 2, 1: 4}),  # J0 confiesa, J1 proporciona evidencia
        tuple(sorted([(0, 2), (1, 1)])): Pagos(valores={0: 4, 1: 2}),  # J0 proporciona evidencia, J1 confiesa
        
        tuple(sorted([(0, 2), (1, 2)])): Pagos(valores={0: 1, 1: 1}),  # Ambos proporcionan evidencia
    }
)

# Encontrar equilibrios de Nash
equilibrios = encontrar_equilibrios_nash(dilema_prisionero)

print(f"Juego: {dilema_prisionero.nombre}")
print("Equilibrios de Nash encontrados:")
for i, (estrategias, pagos) in enumerate(equilibrios, 1):
    print(f"\nEquilibrio {i}:")
    for j, estrategia in enumerate(estrategias):
        print(f"  Jugador {j}: {estrategia.nombre}")
    print("  Pagos:")
    for j in range(dilema_prisionero.jugadores):
        print(f"    Jugador {j}: {pagos.valores[j]}")

# Análisis de los equilibrios
print("\nAnálisis de los equilibrios:")

if not equilibrios:
    print("  No se encontraron equilibrios de Nash en este juego.")
else:
    for i, (estrategias, pagos) in enumerate(equilibrios, 1):
        print(f"\n  Equilibrio {i}:")
        estrategias_str = ", ".join([f"Jugador {j}: {e.nombre}" for j, e in enumerate(estrategias)])
        pagos_str = ", ".join([f"Jugador {j}: {p}" for j, p in pagos.valores.items()])
        print(f"  Estrategias: {estrategias_str}")
        print(f"  Pagos: {pagos_str}")
        
        # Análisis específico para este juego
        if all(e.nombre == "Confesar" for e in estrategias):
            print("  Este es el clásico equilibrio del dilema del prisionero: aunque ambos jugadores")
            print("  estarían mejor si cooperaran (callaran), la estrategia dominante es confesar.")
            print("  Este equilibrio es ineficiente desde el punto de vista de Pareto, ya que")
            print("  ambos jugadores podrían mejorar si cambiaran simultáneamente a 'Callar'.")
        elif all(e.nombre == "Proporcionar Evidencia" for e in estrategias):
            print("  En este equilibrio, ambos jugadores eligen proporcionar evidencia, lo que resulta")
            print("  en una situación aún peor que la confesión mutua. Esto ilustra cómo la introducción")
            print("  de opciones adicionales puede empeorar los resultados en juegos no cooperativos.")

Juego: Dilema del Prisionero Modificado
Equilibrios de Nash encontrados:

Equilibrio 1:
  Jugador 0: Confesar
  Jugador 1: Proporcionar Evidencia
  Pagos:
    Jugador 0: 2.0
    Jugador 1: 4.0

Equilibrio 2:
  Jugador 0: Proporcionar Evidencia
  Jugador 1: Confesar
  Pagos:
    Jugador 0: 4.0
    Jugador 1: 2.0

Análisis de los equilibrios:

  Equilibrio 1:
  Estrategias: Jugador 0: Confesar, Jugador 1: Proporcionar Evidencia
  Pagos: Jugador 0: 2.0, Jugador 1: 4.0

  Equilibrio 2:
  Estrategias: Jugador 0: Proporcionar Evidencia, Jugador 1: Confesar
  Pagos: Jugador 0: 4.0, Jugador 1: 2.0


In [19]:
print(dilema_prisionero.estrategias)

{0: [Estrategia(nombre='Callar', descripcion='Mantener silencio y no confesar el crimen'), Estrategia(nombre='Confesar', descripcion='Confesar el crimen e implicar al otro sospechoso'), Estrategia(nombre='Proporcionar Evidencia', descripcion='Ofrecer evidencia que podría reducir la propia sentencia pero aumentar la del otro')], 1: [Estrategia(nombre='Callar', descripcion='Mantener silencio y no confesar el crimen'), Estrategia(nombre='Confesar', descripcion='Confesar el crimen e implicar al otro sospechoso'), Estrategia(nombre='Proporcionar Evidencia', descripcion='Ofrecer evidencia que podría reducir la propia sentencia pero aumentar la del otro')]}


## Análisis de la Estructura del Juego

Esta definición del juego tiene varias características importantes:

- **Metadatos**:
  - `nombre` y `descripcion`: Proporcionan contexto y narrativa para el juego
  - `jugadores`: Establece que hay 2 países tomando decisiones
  
- **Estrategias**:
  - Cada país (jugador) tiene las mismas tres estrategias disponibles
  - Las estrategias representan diferentes enfoques de política medioambiental

- **Matriz de Pagos**: 
  - Define recompensas para cada combinación de estrategias
  - Los pagos representan "bienestar nacional" (valores más altos son mejores)
  - La estructura de pagos refleja varios aspectos realistas de políticas ambientales:

1. **Análisis de escenarios clave**:
   
   - **Ambos contaminan (2, 2)**: Resultado mutuamente perjudicial (como el choque en el juego del gallina clásico)
   
   - **Uno contamina, otro regula (10, 0) o (0, 10)**: El que contamina aprovecha la responsabilidad del otro (asimetría extrema)
   
   - **Ambos regulan (6, 6)**: Resultado moderadamente bueno de cooperación parcial
   
   - **Tecnología verde vs. Contaminar (3, 8) o (8, 3)**: El innovador sufre ante el contaminador
   
   - **Tecnología verde vs. Regular (7, 5) o (5, 7)**: Buena sinergia entre regulación e innovación
   
   - **Ambos tecnología verde (9, 9)**: El mejor resultado colectivo posible

- **¿Por qué estos valores específicos?**
  - Preservan la estructura básica del juego del gallina (peor resultado si ambos eligen la estrategia agresiva)
  - Añaden un componente de "bien público" donde la cooperación beneficia a todos
  - Crean una tercera opción (tecnología verde) que es superior a las opciones tradicionales
  - Modelan las tensiones entre intereses nacionales a corto plazo y bienestar global a largo plazo

## Análisis de Equilibrios de Nash

La sección de código para encontrar y analizar equilibrios:

1. Utiliza la función `encontrar_equilibrios_nash` para identificar todos los equilibrios estables
2. Imprime información detallada de cada equilibrio (estrategias y pagos resultantes)
3. Proporciona un análisis específico para cada tipo de equilibrio encontrado

## Tipos de Equilibrios y Sus Interpretaciones

El código analiza específicamente varios tipos posibles de equilibrios:

1. **Equilibrio de cooperación total** (ambos eligen tecnología verde):
   - Pagos: (9, 9) - Resultado óptimo de Pareto
   - Representa la visión de un futuro sostenible con altos niveles de bienestar
   - Requiere confianza mutua y visión a largo plazo

2. **Equilibrios asimétricos** (uno contamina, otro regula):
   - Pagos: (10, 0) o (0, 10) - Gran desigualdad
   - Modelan situaciones reales donde algunos países asumen la carga ambiental mientras otros se benefician
   - Generan tensiones políticas y potenciales conflictos internacionales

3. **Equilibrios mixtos** (tecnología verde y regular):
   - Pagos: (7, 5) o (5, 7) - Distribución moderadamente desigual pero positiva
   - Representan escenarios de transición o especialización
   - Muestran cómo diferentes enfoques pueden complementarse

## Resultados Esperados

Al ejecutar este código, esperaríamos encontrar varios equilibrios de Nash:

1. **Equilibrios asimétricos**: Donde un país contamina y el otro regula
   - Son equilibrios de Nash porque ningún país puede mejorar unilateralmente
   - Reflejan la dinámica clásica del juego del gallina

2. **Potenciales equilibrios adicionales**: 
   - Dependiendo de los valores exactos de la matriz de pagos, podríamos encontrar equilibrios como:
     - Ambos países adoptan tecnología verde
     - Un país adopta tecnología verde y el otro regula

## Implicaciones y Aprendizajes

Este juego modeliza varias dinámicas importantes en políticas ambientales internacionales:

1. **El dilema de la acción climática**: 
   - El incentivo individual para contaminar vs. el beneficio colectivo de la sostenibilidad
   - La tensión entre crecimiento económico a corto plazo y sostenibilidad a largo plazo

2. **Coordinación y consenso internacional**:
   - Muestra por qué los acuerdos vinculantes son necesarios
   - Ilustra el problema de los "free riders" (países que se benefician sin asumir costos)

3. **Innovación como solución**:
   - La tecnología verde como camino para superar el dilema clásico
   - Cómo la innovación puede crear nuevos equilibrios más beneficiosos

Este modelo puede utilizarse para analizar y explicar:
- Negociaciones climáticas internacionales
- Políticas de desarrollo sostenible
- Tensiones entre países desarrollados y en desarrollo
- El papel de la innovación tecnológica en la transición ecológica

La estructura de este juego captura elementos cruciales del debate político actual sobre cambio climático y desarrollo sostenible, mostrando cómo la teoría de juegos puede iluminar dilemas sociales complejos.
# Explicación del Archivo `juego_chicken.py`

Este archivo implementa una variante medioambiental del clásico "Juego del Gallina" (Chicken) utilizando nuestro framework de teoría de juegos. Vamos a analizar en detalle cada componente del código y su significado.

## Imports Iniciales

```python
from estructuras_juego import EstructuraDeJuego, Estrategia, Pagos, encontrar_equilibrios_nash
```

- Importamos las clases y funciones de nuestro marco de teoría de juegos
- Esto nos permite definir estrategias, la estructura del juego y analizar equilibrios

## Definición de Estrategias

```python
contaminar = Estrategia(
    nombre="Contaminar", 
    descripcion="Priorizar el crecimiento económico sin restricciones ambientales"
)

regular = Estrategia(
    nombre="Regular", 
    descripcion="Implementar regulaciones estrictas para reducir la contaminación"
)

tecnologia_verde = Estrategia(
    nombre="Tecnología Verde", 
    descripcion="Invertir en tecnologías limpias y desarrollo sostenible"
)
```

Aquí definimos tres estrategias para los jugadores, que representan diferentes enfoques de política ambiental:

1. **Contaminar**: Representa la estrategia agresiva y no cooperativa (equivalente a "no ceder" en el juego del gallina)
2. **Regular**: Representa una postura intermedia (ceder parcialmente)
3. **Tecnología Verde**: Representa una estrategia innovadora que busca soluciones sostenibles

- **¿Por qué estas estrategias?**
  - Trasladar el clásico juego del gallina a un contexto medioambiental contemporáneo
  - Añadir una tercera opción que rompe el binario tradicional "ceder/no ceder"
  - Modelar un escenario relevante para políticas públicas actuales

## Creación de la Estructura del Juego

```python
chicken_ambiental = EstructuraDeJuego(
    nombre="Chicken Medioambiental",
    descripcion="""
    Dos países industrializados enfrentan decisiones sobre sus políticas ambientales y económicas.
    Cada país tiene tres opciones:
    1. Contaminar: Priorizar el crecimiento económico sin restricciones ambientales
    2. Regular: Implementar regulaciones estrictas para reducir la contaminación
    3. Tecnología Verde: Invertir en tecnologías limpias y desarrollo sostenible
    
    Los pagos representan bienestar nacional (valores más altos son mejores):
    - Si ambos contaminan, ambos sufren graves consecuencias ambientales (2 unidades cada uno)
    - Si uno contamina y el otro regula, el que contamina obtiene grandes beneficios económicos (10 unidades)
      mientras que el que regula asume los costos y recibe pocos beneficios (0 unidades)
    - Si ambos regulan, ambos obtienen beneficios moderados y costos compartidos (6 unidades cada uno)
    - Si uno contamina y el otro elige tecnología verde, el que contamina obtiene beneficios buenos (8 unidades)
      y el de tecnología verde obtiene beneficios bajos (3 unidades)
    - Si uno regula y el otro elige tecnología verde, el que regula obtiene beneficios moderados (5 unidades)
      y el de tecnología verde obtiene beneficios buenos (7 unidades)
    - Si ambos eligen tecnología verde, ambos obtienen beneficios altos sostenibles (9 unidades cada uno)
    """,
    jugadores=2,
    estrategias={
        0: [contaminar, regular, tecnologia_verde],
        1: [contaminar, regular, tecnologia_verde]
    },
    matriz_pagos={
        # (jugador, índice_estrategia)
        tuple(sorted([(0, 0), (1, 0)])): Pagos(valores={0: 2, 1: 2    }
)

# Encontrar equilibrios de Nash
equilibrios = encontrar_equilibrios_nash(chicken_ambiental)

print(f"Juego: {chicken_ambiental.nombre}")
print("Equilibrios de Nash encontrados:")
for i, (estrategias, pagos) in enumerate(equilibrios, 1):
    print(f"\nEquilibrio {i}:")
    for j, estrategia in enumerate(estrategias):
        print(f"  Jugador {j}: {estrategia.nombre}")
    print("  Pagos:")
    for j in range(chicken_ambiental.jugadores):
        print(f"    Jugador {j}: {pagos.valores[j]}")

# Análisis de los equilibrios
print("\nAnálisis de los equilibrios:")

if not equilibrios:
    print("  No se encontraron equilibrios de Nash en este juego.")
else:
    for i, (estrategias, pagos) in enumerate(equilibrios, 1):
        print(f"\n  Equilibrio {i}:")
        estrategias_str = ", ".join([f"Jugador {j}: {e.nombre}" for j, e in enumerate(estrategias)])
        pagos_str = ", ".join([f"Jugador {j}: {p}" for j, p in pagos.valores.items()])
        print(f"  Estrategias: {estrategias_str}")
        print(f"  Pagos: {pagos_str}")
        
        # Análisis específico para este juego
        if estrategias[0].nombre == "Tecnología Verde" and estrategias[1].nombre == "Tecnología Verde":
            print("  Este equilibrio representa la cooperación ideal donde ambos países")
            print("  invierten en tecnologías verdes, obteniendo el mayor beneficio conjunto posible.")
            print("  Es el resultado más eficiente según Pareto, siendo socialmente óptimo.")
        elif estrategias[0].nombre == "Contaminar" and estrategias[1].nombre == "Regular":
            print("  En este equilibrio asimétrico, un país explota la situación contaminando")
            print("  mientras el otro asume la responsabilidad de regular. Aunque es un equilibrio,")
            print("  genera una distribución injusta de costos y beneficios, creando tensiones políticas.")
        elif estrategias[0].nombre == "Regular" and estrategias[1].nombre == "Contaminar":
            print("  Este es el equilibrio asimétrico inverso, donde el segundo país explota")
            print("  la situación mientras el primero asume los costos. La asimetría crea")
            print("  incentivos para renegociar o buscar acuerdos internacionales vinculantes.")
        elif estrategias[0].nombre == "Tecnología Verde" and estrategias[1].nombre == "Regular":
            print("  En este equilibrio, un país adopta tecnologías verdes avanzadas mientras")
            print("  el otro se limita a regular. Representa una situación de liderazgo tecnológico")
            print("  que podría ser transitoria hacia la adopción completa de tecnologías verdes.")
        elif estrategias[0].nombre == "Regular" and estrategias[1].nombre == "Tecnología Verde":
            print("  Situación inversa al equilibrio anterior, donde el segundo país lidera")
            print("  la innovación verde mientras el primero se enfoca en la regulación."),  # Ambos contaminan
        tuple(sorted([(0, 0), (1, 1)])): Pagos(valores={0: 10, 1: 0}),  # J0 contamina, J1 regula
        tuple(sorted([(0, 1), (1, 0)])): Pagos(valores={0: 0, 1: 10}),  # J0 regula, J1 contamina
        tuple(sorted([(0, 1), (1, 1)])): Pagos(valores={0: 6, 1: 6}),  # Ambos regulan
        
        tuple(sorted([(0, 0), (1, 2)])): Pagos(valores={0: 8, 1: 3}),  # J0 contamina, J1 tecnología verde
        tuple(sorted([(0, 2), (1, 0)])): Pagos(valores={0: 3, 1: 8}),  # J0 tecnología verde, J1 contamina
        
        tuple(sorted([(0, 1), (1, 2)])): Pagos(valores={0: 5, 1: 7}),  # J0 regula, J1 tecnología verde
        tuple(sorted([(0, 2), (1, 1)])): Pagos(valores={0: 7, 1: 5}),  # J0 tecnología verde, J1 regula
        
        tuple(sorted([(0, 2), (1, 2)])): Pagos(valores={0: 9, 1: 9}),  # Ambos tecnología verde
    }

# Codigo

In [4]:
# Definición del juego: Chicken Medioambiental
# En esta versión, dos países industrializados enfrentan decisiones sobre políticas ambientales.
# Cada país puede elegir entre contaminar (obtener beneficios económicos a corto plazo) o
# implementar regulaciones estrictas (asumir costos a corto plazo para beneficios a largo plazo).
# Además, existe una tercera opción: adoptar tecnologías verdes, que tiene costos moderados
# pero beneficios sostenibles.

# Definimos las estrategias para ambos jugadores
contaminar = Estrategia(
    nombre="Contaminar", 
    descripcion="Priorizar el crecimiento económico sin restricciones ambientales"
)

regular = Estrategia(
    nombre="Regular", 
    descripcion="Implementar regulaciones estrictas para reducir la contaminación"
)

tecnologia_verde = Estrategia(
    nombre="Tecnología Verde", 
    descripcion="Invertir en tecnologías limpias y desarrollo sostenible"
)

# Creamos la estructura del juego
chicken_ambiental = EstructuraDeJuego(
    nombre="Chicken Medioambiental",
    descripcion="""
    Dos países industrializados enfrentan decisiones sobre sus políticas ambientales y económicas.
    Cada país tiene tres opciones:
    1. Contaminar: Priorizar el crecimiento económico sin restricciones ambientales
    2. Regular: Implementar regulaciones estrictas para reducir la contaminación
    3. Tecnología Verde: Invertir en tecnologías limpias y desarrollo sostenible
    
    Los pagos representan bienestar nacional (valores más altos son mejores):
    - Si ambos contaminan, ambos sufren graves consecuencias ambientales (2 unidades cada uno)
    - Si uno contamina y el otro regula, el que contamina obtiene grandes beneficios económicos (10 unidades)
      mientras que el que regula asume los costos y recibe pocos beneficios (0 unidades)
    - Si ambos regulan, ambos obtienen beneficios moderados y costos compartidos (6 unidades cada uno)
    - Si uno contamina y el otro elige tecnología verde, el que contamina obtiene beneficios buenos (8 unidades)
      y el de tecnología verde obtiene beneficios bajos (3 unidades)
    - Si uno regula y el otro elige tecnología verde, el que regula obtiene beneficios moderados (5 unidades)
      y el de tecnología verde obtiene beneficios buenos (7 unidades)
    - Si ambos eligen tecnología verde, ambos obtienen beneficios altos sostenibles (9 unidades cada uno)
    """,
    jugadores=2,
    estrategias={
        0: [contaminar, regular, tecnologia_verde],
        1: [contaminar, regular, tecnologia_verde]
    },
    matriz_pagos={
        # (jugador, índice_estrategia)
        tuple(sorted([(0, 0), (1, 0)])): Pagos(valores={0: 2, 1: 2}),  # Ambos contaminan
        tuple(sorted([(0, 0), (1, 1)])): Pagos(valores={0: 10, 1: 0}),  # J0 contamina, J1 regula
        tuple(sorted([(0, 1), (1, 0)])): Pagos(valores={0: 0, 1: 10}),  # J0 regula, J1 contamina
        tuple(sorted([(0, 1), (1, 1)])): Pagos(valores={0: 6, 1: 6}),  # Ambos regulan
        
        tuple(sorted([(0, 0), (1, 2)])): Pagos(valores={0: 8, 1: 3}),  # J0 contamina, J1 tecnología verde
        tuple(sorted([(0, 2), (1, 0)])): Pagos(valores={0: 3, 1: 8}),  # J0 tecnología verde, J1 contamina
        
        tuple(sorted([(0, 1), (1, 2)])): Pagos(valores={0: 5, 1: 7}),  # J0 regula, J1 tecnología verde
        tuple(sorted([(0, 2), (1, 1)])): Pagos(valores={0: 7, 1: 5}),  # J0 tecnología verde, J1 regula
        
        tuple(sorted([(0, 2), (1, 2)])): Pagos(valores={0: 9, 1: 9}),  # Ambos tecnología verde
    }
)

# Encontrar equilibrios de Nash
equilibrios = encontrar_equilibrios_nash(chicken_ambiental)

print(f"Juego: {chicken_ambiental.nombre}")
print("Equilibrios de Nash encontrados:")
for i, (estrategias, pagos) in enumerate(equilibrios, 1):
    print(f"\nEquilibrio {i}:")
    for j, estrategia in enumerate(estrategias):
        print(f"  Jugador {j}: {estrategia.nombre}")
    print("  Pagos:")
    for j in range(chicken_ambiental.jugadores):
        print(f"    Jugador {j}: {pagos.valores[j]}")

# Análisis de los equilibrios
print("\nAnálisis de los equilibrios:")

if not equilibrios:
    print("  No se encontraron equilibrios de Nash en este juego.")
else:
    for i, (estrategias, pagos) in enumerate(equilibrios, 1):
        print(f"\n  Equilibrio {i}:")
        estrategias_str = ", ".join([f"Jugador {j}: {e.nombre}" for j, e in enumerate(estrategias)])
        pagos_str = ", ".join([f"Jugador {j}: {p}" for j, p in pagos.valores.items()])
        print(f"  Estrategias: {estrategias_str}")
        print(f"  Pagos: {pagos_str}")
        
        # Análisis específico para este juego
        if estrategias[0].nombre == "Tecnología Verde" and estrategias[1].nombre == "Tecnología Verde":
            print("  Este equilibrio representa la cooperación ideal donde ambos países")
            print("  invierten en tecnologías verdes, obteniendo el mayor beneficio conjunto posible.")
            print("  Es el resultado más eficiente según Pareto, siendo socialmente óptimo.")
        elif estrategias[0].nombre == "Contaminar" and estrategias[1].nombre == "Regular":
            print("  En este equilibrio asimétrico, un país explota la situación contaminando")
            print("  mientras el otro asume la responsabilidad de regular. Aunque es un equilibrio,")
            print("  genera una distribución injusta de costos y beneficios, creando tensiones políticas.")
        elif estrategias[0].nombre == "Regular" and estrategias[1].nombre == "Contaminar":
            print("  Este es el equilibrio asimétrico inverso, donde el segundo país explota")
            print("  la situación mientras el primero asume los costos. La asimetría crea")
            print("  incentivos para renegociar o buscar acuerdos internacionales vinculantes.")
        elif estrategias[0].nombre == "Tecnología Verde" and estrategias[1].nombre == "Regular":
            print("  En este equilibrio, un país adopta tecnologías verdes avanzadas mientras")
            print("  el otro se limita a regular. Representa una situación de liderazgo tecnológico")
            print("  que podría ser transitoria hacia la adopción completa de tecnologías verdes.")
        elif estrategias[0].nombre == "Regular" and estrategias[1].nombre == "Tecnología Verde":
            print("  Situación inversa al equilibrio anterior, donde el segundo país lidera")
            print("  la innovación verde mientras el primero se enfoca en la regulación.")

Juego: Chicken Medioambiental
Equilibrios de Nash encontrados:

Equilibrio 1:
  Jugador 0: Tecnología Verde
  Jugador 1: Tecnología Verde
  Pagos:
    Jugador 0: 9.0
    Jugador 1: 9.0

Análisis de los equilibrios:

  Equilibrio 1:
  Estrategias: Jugador 0: Tecnología Verde, Jugador 1: Tecnología Verde
  Pagos: Jugador 0: 9.0, Jugador 1: 9.0
  Este equilibrio representa la cooperación ideal donde ambos países
  invierten en tecnologías verdes, obteniendo el mayor beneficio conjunto posible.
  Es el resultado más eficiente según Pareto, siendo socialmente óptimo.


# Explicación del Archivo `juego_coordinacion.py`

Este archivo implementa un juego de coordinación en el contexto de inversiones tecnológicas, utilizando nuestro framework de teoría de juegos. A continuación, analizamos detalladamente el código y su significado.

## Imports Iniciales

```python
from estructuras_juego import EstructuraDeJuego, Estrategia, Pagos, encontrar_equilibrios_nash
```

- Importamos las clases y funciones base que definimos en nuestro marco de teoría de juegos
- Estas importaciones permiten definir estrategias, la estructura del juego y analizar sus equilibrios

## Definición de Estrategias

```python
estandar_a = Estrategia(
    nombre="Estándar A", 
    descripcion="Invertir en el desarrollo del estándar tecnológico A"
)

estandar_b = Estrategia(
    nombre="Estándar B", 
    descripcion="Invertir en el desarrollo del estándar tecnológico B"
)

solucion_hibrida = Estrategia(
    nombre="Solución Híbrida", 
    descripcion="Desarrollar una solución que intenta ser compatible con ambos estándares"
)
```

Aquí definimos tres estrategias disponibles para los jugadores:

1. **Estándar A**: Invertir exclusivamente en un estándar tecnológico específico
2. **Estándar B**: Invertir en un estándar tecnológico alternativo
3. **Solución Híbrida**: Desarrollar una tecnología que busca compatibilidad con ambos estándares

- **¿Por qué estas estrategias?**
  - Representan un escenario común en industrias tecnológicas donde la compatibilidad es crucial
  - Modelan el dilema entre especialización (mayor rendimiento) vs. compatibilidad (menor riesgo)
  - La opción híbrida introduce complejidad realista al modelo básico de coordinación

## Creación de la Estructura del Juego

```python
coordinacion_tecnologica = EstructuraDeJuego(
    nombre="Coordinación de Inversión Tecnológica",
    descripcion="""
    Dos empresas tecnológicas deben decidir en qué estándar invertir para el desarrollo de una nueva tecnología.
    Cada empresa tiene tres opciones:
    1. Estándar A: Invertir exclusivamente en el desarrollo del estándar tecnológico A
    2. Estándar B: Invertir exclusivamente en el desarrollo del estándar tecnológico B
    3. Solución Híbrida: Desarrollar una solución compatible con ambos estándares a un costo adicional
    
    Los pagos representan beneficios empresariales (valores más altos son mejores):
    - Si ambas eligen el mismo estándar (sea A o B), ambas se benefician de la compatibilidad 
      y economías de escala (8 unidades cada una)
    - Si una elige A y la otra B, ambas obtienen beneficios mínimos por falta de compatibilidad 
      (2 unidades cada una)
    - Si una elige un estándar específico y la otra la solución híbrida, la de estándar específico 
      obtiene buenos beneficios (7 unidades) y la de solución híbrida obtiene beneficios moderados 
      por los costos adicionales (5 unidades)
    - Si ambas eligen la solución híbrida, ambas obtienen beneficios moderados por la compatibilidad 
      pero con mayores costos (6 unidades cada una)
    """,
    jugadores=2,
    estrategias={
        0: [estandar_a, estandar_b, solucion_hibrida],
        1: [estandar_a, estandar_b, solucion_hibrida]
    },
    matriz_pagos={
        # (jugador, índice_estrategia)
        tuple(sorted([(0, 0), (1, 0)])): Pagos(valores={0: 8, 1: 8}),  # Ambos estándar A
        tuple(sorted([(0, 0), (1, 1)])): Pagos(valores={0: 2, 1: 2}),  # J0 estándar A, J1 estándar B
        tuple(sorted([(0, 1), (1, 0)])): Pagos(valores={0: 2, 1: 2}),  # J0 estándar B, J1 estándar A
        tuple(sorted([(0, 1), (1, 1)])): Pagos(valores={0: 8, 1: 8}),  # Ambos estándar B
        
        tuple(sorted([(0, 0), (1, 2)])): Pagos(valores={0: 7, 1: 5}),  # J0 estándar A, J1 solución híbrida
        tuple(sorted([(0, 2), (1, 0)])): Pagos(valores={0: 5, 1: 7}),  # J0 solución híbrida, J1 estándar A
        
        tuple(sorted([(0, 1), (1, 2)])): Pagos(valores={0: 7, 1: 5}),  # J0 estándar B, J1 solución híbrida
        tuple(sorted([(0, 2), (1, 1)])): Pagos(valores={0: 5, 1: 7}),  # J0 solución híbrida, J1 estándar B
        
        tuple(sorted([(0, 2), (1, 2)])): Pagos(valores={0: 6, 1: 6}),  # Ambos solución híbrida
    }
)
```

Esta definición completa del juego incluye:

- **Metadatos**:
  - `nombre` y `descripcion`: Proporcionan contexto narrativo sobre el escenario de inversión tecnológica
  - `jugadores`: Especifica que hay 2 empresas tomando decisiones

- **Estrategias**:
  - Cada empresa tiene las mismas tres estrategias disponibles
  - Las estrategias representan diferentes enfoques de inversión tecnológica

- **Matriz de Pagos**:
  - Define los beneficios para cada combinación posible de decisiones
  - Los pagos representan "beneficios empresariales" (valores más altos son mejores)
  
- **Estructura de la matriz de pagos**:

  1. **Escenarios de coordinación exitosa**:
     - Ambos eligen Estándar A: (8, 8) - Beneficios altos por compatibilidad
     - Ambos eligen Estándar B: (8, 8) - Igualmente beneficioso
  
  2. **Escenarios de coordinación fallida**:
     - Uno elige A, otro B: (2, 2) - Grandes pérdidas por incompatibilidad
  
  3. **Escenarios con solución híbrida**:
     - Estándar específico vs. Híbrido: (7, 5) o (5, 7) - El que elige un estándar específico se beneficia más
     - Ambos eligen híbrido: (6, 6) - Resultado moderadamente bueno

- **¿Por qué estos valores de pago?**
  - Reflejan las economías de escala y efectos de red en industrias tecnológicas
  - Modelan el trade-off entre especialización y compatibilidad
  - Premian la coordinación pero también ofrecen una opción "segura" (híbrida)
  - Crean múltiples equilibrios de Nash con diferentes implicaciones

## Análisis de Equilibrios de Nash

```python
# Encontrar equilibrios de Nash
equilibrios = encontrar_equilibrios_nash(coordinacion_tecnologica)

print(f"Juego: {coordinacion_tecnologica.nombre}")
print("Equilibrios de Nash encontrados:")
for i, (estrategias, pagos) in enumerate(equilibrios, 1):
    print(f"\nEquilibrio {i}:")
    for j, estrategia in enumerate(estrategias):
        print(f"  Jugador {j}: {estrategia.nombre}")
    print("  Pagos:")
    for j in range(coordinacion_tecnologica.jugadores):
        print(f"    Jugador {j}: {pagos.valores[j]}")
```

Esta parte del código:
1. Utiliza la función `encontrar_equilibrios_nash` para identificar todos los equilibrios del juego
2. Imprime cada equilibrio encontrado, mostrando:
   - Las estrategias elegidas por cada jugador
   - Los pagos resultantes para cada jugador

## Análisis e Interpretación de Resultados

```python
# Análisis de los equilibrios
print("\nAnálisis de los equilibrios:")

if not equilibrios:
    print("  No se encontraron equilibrios de Nash en este juego.")
else:
    for i, (estrategias, pagos) in enumerate(equilibrios, 1):
        print(f"\n  Equilibrio {i}:")
        estrategias_str = ", ".join([f"Jugador {j}: {e.nombre}" for j, e in enumerate(estrategias)])
        pagos_str = ", ".join([f"Jugador {j}: {p}" for j, p in pagos.valores.items()])
        print(f"  Estrategias: {estrategias_str}")
```

Esta sección proporciona un análisis de los equilibrios encontrados:
1. Verifica si se encontraron equilibrios
2. Para cada equilibrio:
   - Muestra un resumen de las estrategias y pagos
   - (El código está truncado y no incluye análisis específicos para cada tipo de equilibrio)

## Resultados Esperados

Al ejecutar este código, esperaríamos encontrar varios equilibrios de Nash:

1. **Equilibrios de coordinación**:
   - Ambas empresas eligen Estándar A: (8, 8)
   - Ambas empresas eligen Estándar B: (8, 8)
   
   Estos son los equilibrios clásicos de coordinación. Una vez que ambas empresas acuerdan un estándar, ninguna tiene incentivo para desviarse unilateralmente.

2. **Potencial equilibrio híbrido**:
   - Ambas empresas eligen Solución Híbrida: (6, 6)
   
   Dependiendo de los valores exactos, este podría ser un equilibrio adicional. Representa una "solución segura" donde ambas empresas minimizan el riesgo.

## Análisis Completo del Juego

### Características Principales

Este juego modeliza varias dinámicas importantes en decisiones de inversión tecnológica:

1. **Problema de coordinación**:
   - Hay múltiples equilibrios igualmente buenos (tanto A como B son viables)
   - El desafío es coordinar para llegar al mismo equilibrio

2. **Riesgo de descoordinación**:
   - Elegir estándares diferentes resulta en grandes pérdidas para ambas partes
   - Esto crea tensión entre competir por el estándar propio y asegurar compatibilidad

3. **Estrategia de cobertura**:
   - La opción híbrida representa una "póliza de seguro" contra la descoordinación
   - Sacrifica algo de rendimiento para reducir el riesgo

### Implicaciones para el Mundo Real

Este modelo captura elementos clave de muchas decisiones tecnológicas:

1. **Guerras de estándares**:
   - VHS vs. Betamax, Blu-ray vs. HD-DVD, diversos protocolos de internet
   - La necesidad de coordinación entre actores independientes

2. **Compatibilidad retroactiva**:
   - Desarrollo de soluciones compatibles con múltiples estándares
   - El costo adicional frente al beneficio de mayor alcance

3. **Efectos de red y economías de escala**:
   - El valor de un estándar crece con su adopción
   - La importancia del "momento" en la adopción de tecnología

### Aplicaciones Prácticas

Este juego puede utilizarse para:
- Analizar decisiones de inversión en tecnologías emergentes
- Entender dinámicas de mercado en industrias con efectos de red
- Evaluar estrategias de compatibilidad y especialización
- Informar políticas de estandarización y regulación tecnológica

La estructura de este juego refleja fielmente los dilemas estratégicos que enfrentan las empresas en entornos de rápida evolución tecnológica, donde la coordinación y compatibilidad son tan importantes como la innovación.

# Codigo

In [5]:
# Definición del juego: Coordinación de Inversión Tecnológica
# En este juego, dos empresas tecnológicas deben decidir en qué estándar invertir.
# El beneficio de cada empresa depende no solo de su decisión sino también de la compatibilidad
# con la decisión de la otra empresa. Además, existe una opción de desarrollar una solución híbrida.

# Definimos las estrategias para ambos jugadores
estandar_a = Estrategia(
    nombre="Estándar A", 
    descripcion="Invertir en el desarrollo del estándar tecnológico A"
)

estandar_b = Estrategia(
    nombre="Estándar B", 
    descripcion="Invertir en el desarrollo del estándar tecnológico B"
)

solucion_hibrida = Estrategia(
    nombre="Solución Híbrida", 
    descripcion="Desarrollar una solución que intenta ser compatible con ambos estándares"
)

# Creamos la estructura del juego
coordinacion_tecnologica = EstructuraDeJuego(
    nombre="Coordinación de Inversión Tecnológica",
    descripcion="""
    Dos empresas tecnológicas deben decidir en qué estándar invertir para el desarrollo de una nueva tecnología.
    Cada empresa tiene tres opciones:
    1. Estándar A: Invertir exclusivamente en el desarrollo del estándar tecnológico A
    2. Estándar B: Invertir exclusivamente en el desarrollo del estándar tecnológico B
    3. Solución Híbrida: Desarrollar una solución compatible con ambos estándares a un costo adicional
    
    Los pagos representan beneficios empresariales (valores más altos son mejores):
    - Si ambas eligen el mismo estándar (sea A o B), ambas se benefician de la compatibilidad 
      y economías de escala (8 unidades cada una)
    - Si una elige A y la otra B, ambas obtienen beneficios mínimos por falta de compatibilidad 
      (2 unidades cada una)
    - Si una elige un estándar específico y la otra la solución híbrida, la de estándar específico 
      obtiene buenos beneficios (7 unidades) y la de solución híbrida obtiene beneficios moderados 
      por los costos adicionales (5 unidades)
    - Si ambas eligen la solución híbrida, ambas obtienen beneficios moderados por la compatibilidad 
      pero con mayores costos (6 unidades cada una)
    """,
    jugadores=2,
    estrategias={
        0: [estandar_a, estandar_b, solucion_hibrida],
        1: [estandar_a, estandar_b, solucion_hibrida]
    },
    matriz_pagos={
        # (jugador, índice_estrategia)
        tuple(sorted([(0, 0), (1, 0)])): Pagos(valores={0: 8, 1: 8}),  # Ambos estándar A
        tuple(sorted([(0, 0), (1, 1)])): Pagos(valores={0: 2, 1: 2}),  # J0 estándar A, J1 estándar B
        tuple(sorted([(0, 1), (1, 0)])): Pagos(valores={0: 2, 1: 2}),  # J0 estándar B, J1 estándar A
        tuple(sorted([(0, 1), (1, 1)])): Pagos(valores={0: 8, 1: 8}),  # Ambos estándar B
        
        tuple(sorted([(0, 0), (1, 2)])): Pagos(valores={0: 7, 1: 5}),  # J0 estándar A, J1 solución híbrida
        tuple(sorted([(0, 2), (1, 0)])): Pagos(valores={0: 5, 1: 7}),  # J0 solución híbrida, J1 estándar A
        
        tuple(sorted([(0, 1), (1, 2)])): Pagos(valores={0: 7, 1: 5}),  # J0 estándar B, J1 solución híbrida
        tuple(sorted([(0, 2), (1, 1)])): Pagos(valores={0: 5, 1: 7}),  # J0 solución híbrida, J1 estándar B
        
        tuple(sorted([(0, 2), (1, 2)])): Pagos(valores={0: 6, 1: 6}),  # Ambos solución híbrida
    }
)

# Encontrar equilibrios de Nash
equilibrios = encontrar_equilibrios_nash(coordinacion_tecnologica)

print(f"Juego: {coordinacion_tecnologica.nombre}")
print("Equilibrios de Nash encontrados:")
for i, (estrategias, pagos) in enumerate(equilibrios, 1):
    print(f"\nEquilibrio {i}:")
    for j, estrategia in enumerate(estrategias):
        print(f"  Jugador {j}: {estrategia.nombre}")
    print("  Pagos:")
    for j in range(coordinacion_tecnologica.jugadores):
        print(f"    Jugador {j}: {pagos.valores[j]}")

# Análisis de los equilibrios
print("\nAnálisis de los equilibrios:")

if not equilibrios:
    print("  No se encontraron equilibrios de Nash en este juego.")
else:
    for i, (estrategias, pagos) in enumerate(equilibrios, 1):
        print(f"\n  Equilibrio {i}:")
        estrategias_str = ", ".join([f"Jugador {j}: {e.nombre}" for j, e in enumerate(estrategias)])
        pagos_str = ", ".join([f"Jugador {j}: {p}" for j, p in pagos.valores.items()])
        print(f"  Estrategias: {estrategias_str}")

Juego: Coordinación de Inversión Tecnológica
Equilibrios de Nash encontrados:

Equilibrio 1:
  Jugador 0: Estándar A
  Jugador 1: Estándar A
  Pagos:
    Jugador 0: 8.0
    Jugador 1: 8.0

Equilibrio 2:
  Jugador 0: Estándar B
  Jugador 1: Estándar B
  Pagos:
    Jugador 0: 8.0
    Jugador 1: 8.0

Análisis de los equilibrios:

  Equilibrio 1:
  Estrategias: Jugador 0: Estándar A, Jugador 1: Estándar A

  Equilibrio 2:
  Estrategias: Jugador 0: Estándar B, Jugador 1: Estándar B
